# Previsão da Taxa de Desemprego no Brasil

Este projeto tem como objetivo analisar e modelar a série histórica da taxa de desemprego brasileira a partir dos dados da PNAD Contínua (IBGE), aplicando técnicas de séries temporais para gerar previsões fora da amostra.

In [ ]:
# Importa bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuração visual
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

In [ ]:
# Importa dados SIDRA
import requests
import pandas as pd

url = "https://apisidra.ibge.gov.br/values/t/6381/n1/all/v/4099/p/all"

response = requests.get(url)

data = response.json()

df = pd.DataFrame(data)

df.head()

In [ ]:
df.columns = df.iloc[0]

# Tira a 1a linha (cabeçalho)
df = df[1:]

df.head()

In [ ]:
# nomes das colunas
df.columns

In [ ]:
# Seleciona período e valor
df_clean = df[["Trimestre Móvel", "Valor"]].copy()

# Renomeia colunas
df_clean.columns = ["trimestre", "taxa_desemprego"]

# Converte taxa p/ número
df_clean["taxa_desemprego"] = (
    df_clean["taxa_desemprego"]
    .str.replace(",", ".")
    .astype(float)
)

df_clean.head()

In [ ]:
# Dicionário p/ converter meses
mapa_meses = {
    "jan": "01", "fev": "02", "mar": "03",
    "abr": "04", "mai": "05", "jun": "06",
    "jul": "07", "ago": "08", "set": "09",
    "out": "10", "nov": "11", "dez": "12"
}

def extrair_data(trimestre_str):
    partes = trimestre_str.split()
    ano = partes[1]
    
    meses = partes[0].split("-")
    ultimo_mes = meses[-1]
    
    mes_num = mapa_meses[ultimo_mes]
    
    return f"{ano}-{mes_num}-01"

# Cria nova coluna de data
df_clean["data"] = df_clean["trimestre"].apply(extrair_data)

# Converte para datetime
df_clean["data"] = pd.to_datetime(df_clean["data"])

df_clean.head()

In [ ]:
# Ordena por data
df_clean = df_clean.sort_values("data")

# Define data como índice
df_clean = df_clean.set_index("data")

# Mantem so a coluna da taxa
serie = df_clean["taxa_desemprego"]

serie.head()

In [ ]:
plt.figure()
serie.plot()
plt.title("Taxa de Desemprego no Brasil")
plt.ylabel("Taxa (%)")
plt.xlabel("Ano")
plt.show()

In [ ]:
# Teste de estacionariedade (ADF)
resultado_adf = adfuller(serie)

print("Estatística ADF:", resultado_adf[0])
print("p-valor:", resultado_adf[1])
print("Valores críticos:")
for chave, valor in resultado_adf[4].items():
    print(f"{chave}: {valor}")

In [ ]:
# Diferenciação da série
serie_diff = serie.diff().dropna()

plt.figure()
serie_diff.plot()
plt.title("Série Diferenciada - Taxa de Desemprego")
plt.show()

In [ ]:
# Teste ADF
resultado_adf_diff = adfuller(serie_diff)

print("Estatística ADF:", resultado_adf_diff[0])
print("p-valor:", resultado_adf_diff[1])

In [ ]:
# Aplicando 2a diferenciação
serie_diff2 = serie.diff().diff().dropna()

# Teste ADF na série c/ 2a diferença
resultado_adf_diff2 = adfuller(serie_diff2)

print("Estatística ADF:", resultado_adf_diff2[0])
print("p-valor:", resultado_adf_diff2[1])

In [ ]:
# Separo últimos 8 períodos para teste
train = serie.iloc[:-8]
test = serie.iloc[-8:]

print("Tamanho treino:", len(train))
print("Tamanho teste:", len(test))

In [ ]:
# Ajustando modelo ARIMA(1,1,1)

modelo = ARIMA(train, order=(1, 1, 1))
resultado = modelo.fit()

print(resultado.summary())

In [ ]:
# Gera previsão para o período de teste
forecast = resultado.forecast(steps=8)

# Visualiza previsões
forecast

In [ ]:
# Cálculo das métricas
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))

print("MAE:", mae)
print("RMSE:", rmse)

In [ ]:
# Modelo ingenuo: repete último valor do treino
naive_forecast = np.repeat(train.iloc[-1], 8)

# Métricas do modelo ingênuo
mae_naive = mean_absolute_error(test, naive_forecast)
rmse_naive = np.sqrt(mean_squared_error(test, naive_forecast))

print("MAE naive:", mae_naive)
print("RMSE naive:", rmse_naive)

In [ ]:
plt.figure()

plt.plot(train.index, train, label="Treino")
plt.plot(test.index, test, label="Real")
plt.plot(test.index, forecast, label="Previsto ARIMA")
plt.plot(test.index, naive_forecast, label="Modelo Ingênuo", linestyle="--")

plt.title("Comparação: Real vs ARIMA vs Modelo Ingênuo")
plt.legend()
plt.show()

In [ ]:
# Analise de autocorrelação (ACF)
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(serie, lags=36)
plt.show()

In [ ]:
# diagnóstico dos resíduos
residuos = resultado.resid

plt.figure()
residuos.plot()
plt.title("Resíduos do Modelo ARIMA")
plt.show()

In [ ]:
plot_acf(residuos, lags=36)
plt.show()

Conclusões

Este estudo analisou a série histórica da taxa de desemprego brasileira utilizando dados da PNAD Contínua (IBGE).

A série apresentou não estacionariedade, confirmada pelo teste ADF (p-valor > 0.05). Após aplicação de diferenciação de primeira ordem, a estacionariedade foi parcialmente atingida, sendo plenamente confirmada após segunda diferenciação.

Optou-se pela especificação ARIMA(1,1,1), considerando:

Interpretação econômica da série

Parcimônia do modelo

Diagnóstico adequado dos resíduos

O modelo apresentou:

MAE ≈ 0.34 p.p.

Desempenho superior ao modelo ingênuo (MAE ≈ 1.04 p.p.)

Resíduos com comportamento compatível com ruído branco

Conclui-se que o modelo captura adequadamente a dinâmica temporal da taxa de desemprego e produz previsões significativamente melhores que abordagens ingênuas.